In [23]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import requests
import re

C:\Users\greg.shoda\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\greg.shoda\Anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [3]:
# Define the endpoint URL for the Socrata Discovery API
url = "https://api.us.socrata.com/api/catalog/v1"

# Set the limit of records per request (100 is the typical max for Socrata)
limit = 100
offset = 0
all_datasets = []

# Loop to paginate through all available datasets
while True:
    # Define parameters with limit and offset
    params = {
        "only": "datasets",
        "limit": limit,
        "offset": offset
    }
    
    # Make the GET request
    response = requests.get(url, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        
        # Extract dataset information
        datasets = data.get("results", [])
        
        # If no more results, break the loop
        if not datasets:
            break
        
        # Append data to the main list
        for dataset in datasets:
            all_datasets.append({
                "name": dataset['resource']['name'],
                "link": dataset['permalink'],
                "description": dataset['resource'].get('description', 'No description available'),
                "domain": dataset['metadata'].get('domain'),  # Optional domain info for city/county identification
                "data_columns": dataset['resource']['columns_name'],
                "data_columns_types": dataset['resource']['columns_datatype'],
                "data_columns_description": dataset['resource']['columns_description']
            })
        
        # Update the offset to fetch the next batch
        offset += limit
    else:
        print(f"Failed to retrieve datasets: {response.status_code}")
        break

# Create a DataFrame with all datasets
df = pd.DataFrame(all_datasets)

# Display the first few rows
print("Total Datasets Retrieved:", len(df))

Failed to retrieve datasets: 400
Total Datasets Retrieved: 10000


In [4]:
state_codes_pattern = r'\b(AL|AK|AZ|AR|CA|CO|CT|DE|FL|GA|HI|ID|IL|IN|IA|KS|KY|LA|ME|MD|MA|MI|MN|MS|MO|MT|NE|NV|NH|NJ|NM|NY|NC|ND|OH|OK|OR|PA|RI|SC|SD|TN|TX|UT|VT|VA|WA|WV|WI|WY)\b'


In [5]:
def extract_state(domain):
    match = re.search(state_codes_pattern, domain, re.IGNORECASE)
    return match.group(0).upper() if match else None

In [6]:
df['potential_state_code'] = df['domain'].apply(extract_state)

In [7]:
def clean_text(text):
    if isinstance(text, str):
        # Remove non-printable characters
        return re.sub(r'[\x00-\x1F\x7F]', '', text)
    return text

In [8]:
df = df.applymap(clean_text)

C:\Users\greg.shoda\AppData\Local\Temp\ipykernel_11904\2197956661.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(clean_text)


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   name                      10000 non-null  object
 1   link                      10000 non-null  object
 2   description               10000 non-null  object
 3   domain                    10000 non-null  object
 4   data_columns              10000 non-null  object
 5   data_columns_types        10000 non-null  object
 6   data_columns_description  10000 non-null  object
 7   potential_state_code      3893 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


In [10]:
df.head()

,name,link,description,domain,data_columns,data_columns_types,data_columns_description,potential_state_code
0,Dallas Police Active Calls,https://www.dallasopendata.com/d/9fxf-t2tr,<b><p>Due to technical issues the Active Calls...,www.dallasopendata.com,"[Unit Num, Nature of Call, Block, Time, Beat, ...","[Text, Text, Text, Text, Text, Text, Text, Tex...","[Responding officers element (unit) number, Pr...",None
1,Lottery Cash 4 Life Winning Numbers: Beginning...,https://data.ny.gov/d/kwxv-fwze,Go to http://on.ny.gov/1xRIvPz on the New York...,data.ny.gov,"[Cash Ball, Draw Date, Winning Numbers]","[Text, Calendar date, Text]","[Cash ball, Draw date, Winning numbers]",NY
2,Provisional COVID-19 Deaths by Sex and Age,https://data.cdc.gov/d/9bhg-hcku,"Effective September 27, 2023, this dataset wil...",data.cdc.gov,"[End Date, Influenza Deaths, Total Deaths, Age...","[Calendar date, Number, Number, Text, Text, Ca...","[Last date of data period, Influenza Deaths (I...",None
3,Howard County Police Department Call For Servi...,https://opendata.howardcountymd.gov/d/qccx-65fg,Calls for Service by computer aided dispatch (...,opendata.howardcountymd.gov,"[Date_Reported, Computer_Aided_Dispatch_Event_...","[Calendar date, Text, Text, Number, Text, Text]","[, , , , , ]",None
4,Health Care Provider Credential Data,https://data.wa.gov/d/qxh8-f4bd,The Washington State Department of Health pres...,data.wa.gov,"[Status, MiddleName, BirthYear, ActionTaken, L...","[Text, Text, Text, Text, Text, Text, Text, Tex...","[, , , , , , , , , , , ]",WA


In [11]:
df['api_link'] = df['link'] + ".json"

In [12]:
df['api_link'].head(20)

0       https://www.dallasopendata.com/d/9fxf-t2tr.json
1                  https://data.ny.gov/d/kwxv-fwze.json
2                 https://data.cdc.gov/d/9bhg-hcku.json
3     https://opendata.howardcountymd.gov/d/qccx-65f...
4                  https://data.wa.gov/d/qxh8-f4bd.json
5              https://datahub.hhs.gov/d/rxn6-qnx8.json
6        https://data.cityofnewyork.us/d/8wbx-tsch.json
7        https://data.cityofnewyork.us/d/vx8i-nprf.json
8     https://opendata.howardcountymd.gov/d/kvz2-j5c...
9        https://data.cityofnewyork.us/d/ic3t-wcy2.json
10            https://www.datos.gov.co/d/gt2j-8ykr.json
11                https://data.cdc.gov/d/nr4s-juj3.json
12      https://data.cityofchicago.org/d/xzkq-xp2w.json
13    https://opendata.howardcountymd.gov/d/xvpn-2pn...
14    https://opendata.howardcountymd.gov/d/f362-6fu...
15    https://opendata.howardcountymd.gov/d/8fxg-nyr...
16       https://data.cityofnewyork.us/d/dpec-ucu7.json
17    https://opendata.howardcountymd.gov/d/6uza

In [15]:
df['api_link'] = df['api_link'].str.replace('/d/', '/resource/')

In [16]:
df['api_link']

0       https://www.dallasopendata.com/resource/9fxf-t...
1             https://data.ny.gov/resource/kwxv-fwze.json
2            https://data.cdc.gov/resource/9bhg-hcku.json
3       https://opendata.howardcountymd.gov/resource/q...
4             https://data.wa.gov/resource/qxh8-f4bd.json
                              ...                        
9995     https://data.edmonton.ca/resource/p2tt-8mv9.json
9996         https://data.cdc.gov/resource/8i5t-42wz.json
9997          https://data.ct.gov/resource/4vva-amjy.json
9998    https://opendata.maryland.gov/resource/94gw-yf...
9999      https://gnb.socrata.com/resource/t6hf-gbks.json
Name: api_link, Length: 10000, dtype: object

In [17]:
api_link_1 = df.loc[0, 'api_link']

In [30]:
def fetch_data(api_link, timeout=5):
    try:
        # Attempt to fetch data from the API link with a timeout
        response = requests.get(api_link, timeout=timeout)
        response.raise_for_status()  # Raise an error for unsuccessful status codes
        return response.json()  # Return JSON data from the response
    except requests.exceptions.Timeout:
        print(f"Request to {api_link} timed out after {timeout} seconds.")
        return "Timeout"  # Return None for timeout cases
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {api_link}: {e}")
        return "ErrorFetchingData"  # Return None for other exceptions

In [24]:
from tqdm import tqdm

In [25]:
tqdm.pandas()

In [ ]:
df['data'] = df['api_link'].progress_apply(fetch_data)

  0%|▎                                                                            | 35/10000 [00:21<5:02:04,  1.82s/it]

Request to https://data.cityofnewyork.us/resource/erm2-nwe9.json timed out after 5 seconds.


  1%|▋                                                                            | 87/10000 [01:05<6:08:58,  2.23s/it]

Request to https://datahub.austintexas.gov/resource/3syk-w9eu.json timed out after 5 seconds.


  2%|█▏                                                                          | 150/10000 [01:49<7:37:32,  2.79s/it]

...

 82%|███████████████████████████████████████████████████████████▊             | 8197/10000 [1:50:00<1:00:15,  2.01s/it]

Request to https://www.data.act.gov.au/resource/92fy-xvmy.json timed out after 5 seconds.


 83%|██████████████████████████████████████████████████████████████▍            | 8322/10000 [1:51:26<10:51,  2.58it/s]

Error fetching data from https://stat.montgomerycountymd.gov/resource/fdyk-etfp.json: [Errno Expecting value] 
<!DOCTYPE html>
<html xmlns="http://www.w3.org/1999/xhtml" lang="en" xml:lang="en">
<head id="ctl00_Head1"><title>
	CountyStat Home Page - Office of Management and Budget - Montgomery County, Maryland
</title><meta http-equiv="Content-Type" content="text/html; charset=utf-8" /><meta name="viewport" content="width=device-width, initial-scale=1.0" /><meta http-equiv="X-UA-Compatible" content="IE=Edge" />    
	<script async src="https://www.googletagmanager.com/gtag/js?id=G-K9STEFSE4V"></script>
	<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.2/css/all.min.css" integrity="sha512-z3gLpd7yknf1YoNbCzqRKc4qyor8gaKU1qmn+CShxbuBusANI9QpRohGBreCFkKxLhei6S9CQXFEbbKuqLg0DA==" crossorigin="anonymous" referrerpolicy="no-referrer" /><link rel="stylesheet" href="//maxcdn.bootstrapcdn.com/font-awesome/4.4.0/css/font-awesome.min.css" />
    <script src="/_t

 84%|███████████████████████████████████████████████████████████████▎           | 8439/10000 [1:52:47<27:40,  1.06s/it]